In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import seaborn as sns
import matplotlib.pyplot as plt

#pd.options.display.max_columns = None

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer

# Creamos objects a partir de las clases PorterStemmer y LancasterStemmer




# Reading CSV`s

In [2]:
real = pd.read_csv("True.csv")
fake = pd.read_csv("Fake.csv")

# Assign tags

In [3]:

real["Fake"] = 0 
fake["Fake"] = 1

# Concat dataframes

In [ ]:
df = pd.concat((real,fake))#


In [5]:
df.shape # veo las dimensiones

(44898, 5)

# Shuffling df

In [6]:
df = df.sample(df.shape[0]) # mezcla instancias

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 12711 to 5312
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   Fake     44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [8]:
df.head(1)

,title,text,subject,date,Fake
12711,"Pope singles out Rome's decay, corruption on t...",ROME (Reuters) - Pope Francis lamented the dec...,worldnews,"December 8, 2017",0


In [9]:
data = df[["text","Fake"]]

In [10]:
data.head()

,text,Fake
12711,ROME (Reuters) - Pope Francis lamented the dec...,0
15377,"A transcript from the Friday, July 31, 2015 Ru...",1
8644,WASHINGTON (Reuters) - The top Democrat on the...,0
5806,Newly released body camera footage shows a La ...,1
20044,DUBAI (Reuters) - Yemen s Houthi leader Abdel-...,0


In [11]:
data.reset_index(inplace = True)

# Preprocessig and Tokenization

In [12]:
from RAFA import nlp
from nltk.corpus import stopwords

stopwords_english = stopwords.words('english')
print(stopwords_english[:15]) 

Cargando paquete RAFA.py
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours']


In [13]:
data.rename(columns ={"text":"Original_Text"},inplace = True)

C:\Users\falen\AppData\Local\Temp\ipykernel_19292\1601325091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns ={"text":"Original_Text"},inplace = True)


# PorterStemmer

In [14]:
porter = PorterStemmer()
lancaster=LancasterStemmer()


In [15]:
def clean_text(text):
    # Convertir todo el texto a minúsculas
    text = text.lower()
    # Remover caracteres especiales
    text = re.sub(r'\W', ' ', text)
    # Remover caracteres simples
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remover caracteres simples del inicio
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    # Remover múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Tokenizar el texto
    text = word_tokenize(text)
    # Remover stopwords
    text = [word for word in text if not word in stopwords_english]
    # Unir las palabras
    text = ' '.join(text)
    #stemmer
    porter.stem(text)
    return text

# Aplicar la función de limpieza a la columna 'review_es'
data['text_stemmatizer'] = data['Original_Text'].apply(clean_text)    
    

C:\Users\falen\AppData\Local\Temp\ipykernel_19292\1471720089.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text_stemmatizer'] = data['Original_Text'].apply(clean_text)


,index,Original_Text,Fake,text_stemmatizer
0,12711,ROME (Reuters) - Pope Francis lamented the dec...,0,rome reuters pope francis lamented decay degra...
1,15377,"A transcript from the Friday, July 31, 2015 Ru...",1,transcript friday july 31 2015 rush limbaugh s...
2,8644,WASHINGTON (Reuters) - The top Democrat on the...,0,washington reuters top democrat house represen...
3,5806,Newly released body camera footage shows a La ...,1,newly released body camera footage shows la ju...
4,20044,DUBAI (Reuters) - Yemen s Houthi leader Abdel-...,0,dubai reuters yemen houthi leader abdel malek ...
...,...,...,...,...
44893,17346,"MOSCOW (Reuters) - Ali Larijani, the speaker o...",0,moscow reuters ali larijani speaker iranian pa...
44894,13116,We re not sure who s doing the polling in Amer...,1,sure polling america judging consistently huge...
44895,57,Donald Trump thinks if he rage-tweets about NF...,1,donald trump thinks rage tweets nfl players ch...
44896,558,WASHINGTON (Reuters) - The House of Representa...,0,washington reuters house representatives ethic...


In [17]:
data.head()

,index,Original_Text,Fake,text_stemmatizer
0,12711,ROME (Reuters) - Pope Francis lamented the dec...,0,rome reuters pope francis lamented decay degra...
1,15377,"A transcript from the Friday, July 31, 2015 Ru...",1,transcript friday july 31 2015 rush limbaugh s...
2,8644,WASHINGTON (Reuters) - The top Democrat on the...,0,washington reuters top democrat house represen...
3,5806,Newly released body camera footage shows a La ...,1,newly released body camera footage shows la ju...
4,20044,DUBAI (Reuters) - Yemen s Houthi leader Abdel-...,0,dubai reuters yemen houthi leader abdel malek ...


# SnowballStemmer

In [18]:

from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")



In [19]:
def clean_text(text):
    # Convertir todo el texto a minúsculas
    text = text.lower()
    # Remover caracteres especiales
    text = re.sub(r'\W', ' ', text)
    # Remover caracteres simples
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remover caracteres simples del inicio
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    # Remover múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Tokenizar el texto
    text = word_tokenize(text)
    # Remover stopwords
    text = [word for word in text if not word in stopwords_english]
    # Unir las palabras
    text = ' '.join(text)
    #stemmer
    text = englishStemmer.stem(text)
    return text
data["Text_SnowballStemmer"] = data['Original_Text'].apply(clean_text)

In [51]:
data.sample()

,index,Original_Text,Fake,text_stemmatizer,Text_SnowballStemmer
10656,287,WASHINGTON (Reuters) - U.S. President Donald T...,0,washington reuters president donald trump woul...,washington reuters president donald trump woul...


# Saving  CSV

In [64]:
data.to_csv("data.csv")

# Training a Logistic Regression and SVC model with text_stemmatizer
Using TfidfVectorizer and GridSearch

In [57]:
from sklearn.model_selection import train_test_split

X = data['text_stemmatizer']
y = data['Fake']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [ ]:
"""from sklearn.feature_extraction.text import TfidfTransformer

Tfidf_encoding=TfidfTransformer().fit_transform(countvectorizer_encoding);

pd.DataFrame(Tfidf_encoding.todense(),columns=vectorizer.get_feature_names_out())
"""

# Logistic Regression

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

parameters = {'C': [0.1, 1, 10, 100]}

In [61]:
lr = LogisticRegression(max_iter=1000)

clf = GridSearchCV(lr, parameters, n_jobs=1)
clf.fit(X_train_vect, y_train)

print('Best parameters:', clf.best_params_)
print('Best score:', clf.best_score_)

Best parameters: {'C': 100}
Best score: 0.9932902738059767


# SVM

In [68]:
from sklearn import svm

In [69]:
parameters = {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01]}

sv = svm.SVC()

clf_svm = GridSearchCV(sv, parameters, n_jobs=1, verbose=2, cv=3)
clf_svm.fit(X_train_vect, y_train)

print('Best parameters:', clf_svm.best_params_)
print('Best score:', clf_svm.best_score_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END ...................................C=0.1, gamma=0.1; total time=11.2min
[CV] END ...................................C=0.1, gamma=0.1; total time=11.2min
[CV] END ...................................C=0.1, gamma=0.1; total time=12.5min
[CV] END ..................................C=0.1, gamma=0.01; total time=15.3min
[CV] END ..................................C=0.1, gamma=0.01; total time=15.3min
[CV] END ..................................C=0.1, gamma=0.01; total time=15.2min
[CV] END .....................................C=1, gamma=0.1; total time= 5.0min
[CV] END .....................................C=1, gamma=0.1; total time= 5.0min
[CV] END .....................................C=1, gamma=0.1; total time= 4.9min
[CV] END ....................................C=1, gamma=0.01; total time=10.6min
[CV] END ....................................C=1, gamma=0.01; total time=10.6min
[CV] END ....................................C=1,

In [71]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions_svm = clf_svm.predict(X_test_vect)

print('Accuracy:', accuracy_score(y_test, predictions_svm))
print('Confusion Matrix:\n', confusion_matrix(y_test, predictions_svm))
print('Classification Report:\n', classification_report(y_test, predictions_svm))

Accuracy: 0.994543429844098
Confusion Matrix:
 [[4313   20]
 [  29 4618]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      4333
           1       1.00      0.99      0.99      4647

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



# Training a Logistic Regression and SVC model with SnowballStemmer
Using TfidfVectorizer and GridSearch


In [72]:
X = data['Text_SnowballStemmer']
y = data['Fake']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [75]:
lr = LogisticRegression(max_iter=1000)
parameters = {'C': [0.1, 1, 10, 100]}
clf = GridSearchCV(lr, parameters, n_jobs=1)
clf.fit(X_train_vect, y_train)

print('Best parameters:', clf.best_params_)
print('Best score:', clf.best_score_)

Best parameters: {'C': 100}
Best score: 0.9932345828913667


In [76]:
parameters = {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01]}

sv = svm.SVC()

clf_svm = GridSearchCV(sv, parameters, n_jobs=1, verbose=2, cv=3)
clf_svm.fit(X_train_vect, y_train)

print('Best parameters:', clf_svm.best_params_)
print('Best score:', clf_svm.best_score_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END ...................................C=0.1, gamma=0.1; total time=12.8min
[CV] END ...................................C=0.1, gamma=0.1; total time=13.0min
[CV] END ...................................C=0.1, gamma=0.1; total time=13.9min
[CV] END ..................................C=0.1, gamma=0.01; total time=47.2min
[CV] END ..................................C=0.1, gamma=0.01; total time=50.4min
[CV] END ................................C=0.1, gamma=0.01; total time=1130.6min
[CV] END .....................................C=1, gamma=0.1; total time= 5.7min
[CV] END .....................................C=1, gamma=0.1; total time= 5.7min
[CV] END .....................................C=1, gamma=0.1; total time= 5.7min
[CV] END ....................................C=1, gamma=0.01; total time=14.3min
[CV] END ....................................C=1, gamma=0.01; total time=14.4min
[CV] END ....................................C=1,

In [77]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions_svm = clf_svm.predict(X_test_vect)

print('Accuracy:', accuracy_score(y_test, predictions_svm))
print('Confusion Matrix:\n', confusion_matrix(y_test, predictions_svm))
print('Classification Report:\n', classification_report(y_test, predictions_svm))

Accuracy: 0.9955456570155902
Confusion Matrix:
 [[4317   16]
 [  24 4623]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      4333
           1       1.00      0.99      1.00      4647

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980

